Python API Tutorial - An Introduction to Using APIs

https://www.dataquest.io/blog/python-api-tutorial/

In [1]:
# Import ALL the things!
import requests
import requests.auth
import json
import getpass

In [31]:
# Got very sick of pasting in all the secret stuff that I can't leave in the notebook so...
# While this isn't the most secure option it is better than typing everything in manually with getpass
# ToDo: encryption! input!

with open('workfile.json', 'r') as file:
        data = json.load(file)

key_list = []

for key in data.keys():
    print(key)
    
# Just saw an idea at https://github.com/ideoforms/python-twitter-examples
# if i put all the secrets into config.py (or something like that)
# then i just run import config and reddit_bullshit = config.reddit_bullshit
# config.py looks like reddit_bullshit = "xxxxxxxxxxxxxxx"

reddit_uagent
reddit_uname
reddit_pass
reddit_api
reddit_api_key
W3W_api
shodan_api_key
ip


### Get current position of ISS

In [27]:
# Make a request to get the current (within 5 seconds) position of the ISS from the open notify API
response = requests.get("http://api.open-notify.org/iss-now.json")
response.status_code

200

In [28]:
response.content

b'{"iss_position": {"latitude": "34.6997", "longitude": "-18.9199"}, "message": "success", "timestamp": 1553737188}'

In [29]:
response.json()

{'iss_position': {'latitude': '34.6997', 'longitude': '-18.9199'},
 'message': 'success',
 'timestamp': 1553737188}

### Get details of next pass

In [48]:
# Params to pass to the API.
# This is the Lat/Long of Melbourne, Australia
parameters = {"lat": -37.81, "lon": 144.96}

# Make a get request with parameters
response = requests.get("http://api.open-notify.org/iss-pass.json", params=parameters)
response.status_code

200

In [49]:
response.content

b'{\n  "message": "success", \n  "request": {\n    "altitude": 100, \n    "datetime": 1553939954, \n    "latitude": -37.81, \n    "longitude": 144.96, \n    "passes": 5\n  }, \n  "response": [\n    {\n      "duration": 594, \n      "risetime": 1553966299\n    }, \n    {\n      "duration": 643, \n      "risetime": 1553972058\n    }, \n    {\n      "duration": 540, \n      "risetime": 1553977950\n    }, \n    {\n      "duration": 499, \n      "risetime": 1553983845\n    }, \n    {\n      "duration": 601, \n      "risetime": 1553989656\n    }\n  ]\n}\n'

In [50]:
response.headers

{'Server': 'nginx/1.10.3', 'Date': 'Sat, 30 Mar 2019 09:59:14 GMT', 'Content-Type': 'application/json', 'Content-Length': '521', 'Connection': 'keep-alive', 'Via': '1.1 vegur'}

In [51]:
data = response.json()
data

{'message': 'success',
 'request': {'altitude': 100,
  'datetime': 1553939954,
  'latitude': -37.81,
  'longitude': 144.96,
  'passes': 5},
 'response': [{'duration': 594, 'risetime': 1553966299},
  {'duration': 643, 'risetime': 1553972058},
  {'duration': 540, 'risetime': 1553977950},
  {'duration': 499, 'risetime': 1553983845},
  {'duration': 601, 'risetime': 1553989656}]}

### Get number of people in space

In [36]:
response = requests.get("http://api.open-notify.org/astros.json")
response.status_code

200

In [37]:
data = response.json()
data["number"]

6

There will be more to come as this topic forms appears in a later part of the DataQuest Data Scientist course.

### Reddit, what could go wrong

In my defence at the end of the DQ article they do tell the reader to have a look at reddit, so...

In [38]:
# My initial attempts would return a 429 code, thanks to u/,,xx_th for the quick fix
response = requests.get("http://reddit.com/api/trending_subreddits.json", headers = {'User-agent': 'your bot 0.1'})
response.status_code

200

In [40]:
# From here out any long response strings will be placed in a variable but not printe
# Having huge blocks of json within the notebook as caused issues for me in the past
reddit_headers = response.headers
# reddit_headers

21

In [43]:
reddit_response = response.json()
# reddit_response

Further exploration is limited by my lack of knowledge of oauth and modhashes, this is a problem for another day

### More Reddit starting with OAuth2

I immediatly discovered that Reddit won't let you do much without authentication.

#### Request a token

In [3]:
# Client API data. Apps client ID and apps client secret
client_auth = requests.auth.HTTPBasicAuth(data['reddit_api'], data['reddit_api_key'])

In [4]:
headers = {"User-Agent": data['reddit_uagent']}
post_data = {"grant_type": "password", "username": data['reddit_uname'], "password": data['reddit_pass']}

In [7]:
response = requests.post("https://www.reddit.com/api/v1/access_token", auth=client_auth, data=post_data, headers=headers)
token = response.json()['access_token']
print(response.status_code)

200


In [8]:
headers = {"Authorization": "bearer " + token, "User-Agent": "ClientMcClientface by /u/timbo72"}
response = requests.get("https://oauth.reddit.com/api/v1/me", headers=headers)
print(response.status_code)
my_data = response.json()
#my_data

200


In [9]:
headers = {"Authorization": "bearer " + token, "User-Agent": "ClientMcClientface by /u/timbo72"}
response = requests.get("https://oauth.reddit.com/r/PrequelMemes/random", headers=headers)
print(response.status_code)
prequel_memes = response.json()
#prequel_memes

200


### Now for something completely different

I've been looking at the 'have i been pwned' API.
First off there is a great script written by Dr Mike Pound, lets give it a go.


In [11]:
import hashlib
import sys

try:
    import requests
except ModuleNotFoundError:
    print("### requests library not found ###")
    raise

    
def lookup_pwned_api(pwd):
    """
    Returns hash and number of times password was seen in pwned database.
    
    Args: pwd: password to check
    
    Returns: A (sha1, count) tuple of the SHA-1 hashed password and the number of times it was found in the database.
    
    Raises: RuntimeError: if there was an error trying to fetch the data from the API.
            UnicodeError: if there was and error UTF_encoding the password.
    """
    sha1pwd = hashlib.sha1(pwd.encode('utf-8')).hexdigest().upper()
    head, tail = sha1pwd[:5], sha1pwd[5:]
    url = "https://api.pwnedpasswords.com/range/" + head
    res = requests.get(url)
    if res.status_code != 200:
        raise RuntimeError("Error fetching '{}': {}".format(url, res.status_code))
    hashes = (line.split(":") for line in res.text.splitlines())
    count = next((int(count) for t, count in hashes if t == tail), 0)
    return sha1pwd, count


def main(args):
    ec = 0
    for pwd in args or sys.stdin:
        pwd = pwd.strip()
        try:
            sha1pwd, count = lookup_pwned_api(pwd)
            
            if count:
                foundmsg = "{0} was found with {1} occurances (hash: {2})"
                print(foundmsg.format(pwd, count, sha1pwd))
                ec = 1
            else:
                print("{} was not found".format(pwd))
        except UnicodeError:
            errormsg = sys.exc_info()[1]
            print("{0} could not be checked: {1}". format(pwd, errormsg))
            ec = 1
            continue
    return ec

import getpass
lookup_pwned_api(getpass.getpass("Password"))

Password········


('7CB9DED83CD7F2704D6435C743ED3124ED56C701', 766)

Lets have a play with V2

In [10]:
# Worth noting that this won't work if the run via a VPN (i.e. from my server)

pwd = getpass.getpass("input email/username to check:")
headers = {"api-version":"2", "user-agent":"Timbos-api-project"}    
url = "https://haveibeenpwned.com/api/breachedaccount/" + pwd + "?truncateResponse=true"
response = requests.get(url, headers = headers)
print("status code:", response.status_code)
if response.status_code == 200:
    data = response.json()
    sites = []
    for i in data:
        sites.append(i['Name'])
    print(sites)
elif response.status_code== 403:
    print("403! what did you do?")
else:
    print("Lucky you, nothing found!")

input email/username to check:········
status code: 200
['OnlinerSpambot']


### What3Words

Heard about an interesting idea for geolocation called what3words, they have an API...

In [54]:
key = data['W3W_api']
lat = getpass.getpass("Lat:")
long = getpass.getpass("Long:")
#lat = "-37.810000"
#long = "144.960000"
url = "https://api.what3words.com/v3/convert-to-3wa?coordinates=" + lat + "%2c" + long + "&key=" + key
response = requests.get(url)
response.status_code

Lat:········
Long:········


200

In [55]:
if response.status_code != 200:
    code = response.json()['error']['code']
    message = response.json()['error']['message']
    print(code)
    print(message)
else:
    words = response.json()['words']
    print(words)

wants.bowls.beam


# Shodan.io  
Here we go...

In [34]:
api_key = data['shodan_api_key']
#ip = getpass.getpass("Input an IP to check:")
ip = data['ip']

url = "https://api.shodan.io/shodan/host/{}?key={}".format(ip, api_key)
response = requests.get(url)
response.status_code

200

In [40]:
data = response.json()
data

dict